## Setup


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from rich import print as rprint
from rich.console import Console
from rich import get_console

# Rich setup for Jupyter
console: Console = get_console()
console.is_jupyter = False

In [ ]:
from snap_fit.grid import (
    GridModel,
    GridPos,
    Orientation,
    OrientedPieceType,
    PieceType,
    PlacementState,
    compute_rotation,
    detect_base_orientation,
    get_piece_type,
    get_rotated_edge_pos,
    get_original_edge_pos,
)
from snap_fit.config.types import EdgePos
from snap_fit.data_models.piece_id import PieceId

## 1. Orientation Arithmetic

Test that orientations compose correctly.


In [ ]:
# Basic values
rprint(f"DEG_0 = {Orientation.DEG_0.value}°")
rprint(f"DEG_90 = {Orientation.DEG_90.value}°")
rprint(f"DEG_180 = {Orientation.DEG_180.value}°")
rprint(f"DEG_270 = {Orientation.DEG_270.value}°")

In [ ]:
# Addition (compose rotations)
rprint(f"90° + 90° = {Orientation.DEG_90 + Orientation.DEG_90}")
rprint(f"270° + 180° = {Orientation.DEG_270 + Orientation.DEG_180}")

# Subtraction (inverse rotation)
rprint(f"90° - 90° = {Orientation.DEG_90 - Orientation.DEG_90}")
rprint(f"0° - 90° = {Orientation.DEG_0 - Orientation.DEG_90}")

# Negation
rprint(f"-90° = {-Orientation.DEG_90}")
rprint(f"-270° = {-Orientation.DEG_270}")

In [ ]:
# Steps conversion
for o in Orientation:
    rprint(f"{o.name}: {o.steps} steps")

# Create from steps
rprint(f"\nfrom_steps(1) = {Orientation.from_steps(1)}")
rprint(f"from_steps(5) = {Orientation.from_steps(5)}  # wraps to 1")

## 2. GridPos Model


In [ ]:
# Create positions
pos1 = GridPos(ro=0, co=0)
pos2 = GridPos(ro=2, co=3)

rprint(f"Position 1: {pos1}")
rprint(f"Position 2: {pos2}")

# Hashable - can use in sets/dicts
positions = {pos1, pos2, GridPos(ro=0, co=0)}  # pos1 duplicate
rprint(f"Unique positions in set: {len(positions)}")

# Use as dict key
placement_map = {pos1: "piece_A", pos2: "piece_B"}
rprint(f"Map: {placement_map}")

## 3. PieceType & OrientedPieceType


In [ ]:
# Classify pieces by flat edge count
for flat_count in [0, 1, 2]:
    pt = get_piece_type(flat_count)
    rprint(f"{flat_count} flat edges → {pt.name}")

In [ ]:
# OrientedPieceType combines type + orientation
opt1 = OrientedPieceType(piece_type=PieceType.CORNER, orientation=Orientation.DEG_0)
opt2 = OrientedPieceType(piece_type=PieceType.EDGE, orientation=Orientation.DEG_90)

rprint(f"Corner at canonical: {opt1}")
rprint(f"Edge rotated 90°: {opt2}")

# Hashable
rprint(f"\nHashable: {hash(opt1)}")

## 4. Orientation Detection

Detect a piece's base orientation from its flat edges.


In [ ]:
# Edge piece detection
rprint("[bold]Edge pieces (1 flat):[/bold]")
for edge in EdgePos:
    orientation = detect_base_orientation([edge])
    rprint(f"  Flat on {edge.name}: {orientation.name} ({orientation.value}°)")

In [ ]:
# Corner piece detection
rprint("[bold]Corner pieces (2 flats):[/bold]")
corner_configs = [
    ([EdgePos.TOP, EdgePos.LEFT], "TOP+LEFT (canonical)"),
    ([EdgePos.TOP, EdgePos.RIGHT], "TOP+RIGHT"),
    ([EdgePos.BOTTOM, EdgePos.RIGHT], "BOTTOM+RIGHT"),
    ([EdgePos.BOTTOM, EdgePos.LEFT], "BOTTOM+LEFT"),
]
for edges, desc in corner_configs:
    orientation = detect_base_orientation(edges)
    rprint(f"  {desc}: {orientation.name} ({orientation.value}°)")

## 5. Rotation Computation

Compute the rotation needed to fit a piece into a target slot.


In [ ]:
# Scenario: piece has flat on RIGHT (90°), slot wants flat on BOTTOM (180°)
piece = OrientedPieceType(piece_type=PieceType.EDGE, orientation=Orientation.DEG_90)
target = OrientedPieceType(piece_type=PieceType.EDGE, orientation=Orientation.DEG_180)

rotation = compute_rotation(piece, target)
rprint(f"Piece: {piece}")
rprint(f"Target slot: {target}")
rprint(f"[green]Rotation needed: {rotation.name} ({rotation.value}°)[/green]")

In [ ]:
# Verify: piece.orientation + rotation = target.orientation
result = piece.orientation + rotation
rprint(
    f"Verification: {piece.orientation.value}° + {rotation.value}° = {result.value}°"
)
rprint(f"Matches target? {result == target.orientation}")

## 6. Edge Position Rotation

Track where edges end up after rotation.


In [ ]:
# After 90° rotation, where does each edge go?
rprint("[bold]After 90° rotation:[/bold]")
for edge in EdgePos:
    rotated = get_rotated_edge_pos(edge, Orientation.DEG_90)
    rprint(f"  {edge.name} → {rotated.name}")

In [ ]:
# Inverse: what was originally at each position?
rprint("[bold]What was originally at each position (after 90° rotation)?[/bold]")
for edge in EdgePos:
    original = get_original_edge_pos(edge, Orientation.DEG_90)
    rprint(f"  Now at {edge.name} ← originally {original.name}")

## 7. GridModel

Create a grid and explore its structure.


In [ ]:
# Create a 4x5 grid (matching sample_puzzle_v2: 6 rows × 8 cols, but smaller for demo)
grid = GridModel(rows=4, cols=5)
rprint(f"Grid: {grid}")
rprint(f"Total cells: {grid.total_cells}")
rprint(f"Total edges (for scoring): {grid.total_edges}")

In [ ]:
# Position lists
rprint(f"Corners ({len(grid.corners)}): {grid.corners}")
rprint(f"Edges ({len(grid.edges)}): {grid.edges}")
rprint(f"Inners ({len(grid.inners)}): {grid.inners}")

In [ ]:
# Slot types and required orientations
rprint("[bold]Corner slot requirements:[/bold]")
for pos in grid.corners:
    slot = grid.get_slot_type(pos)
    rprint(f"  {pos}: {slot}")

In [ ]:
# Visualize grid layout
rprint("[bold]Grid Layout (type & orientation):[/bold]")
for ro in range(grid.rows):
    row_str = ""
    for co in range(grid.cols):
        slot = grid.get_slot_type(GridPos(ro=ro, co=co))
        # Short codes: C=Corner, E=Edge, I=Inner, number=orientation/90
        code = slot.piece_type.name[0] + str(slot.orientation.steps)
        row_str += f" {code:>3}"
    rprint(row_str)

In [ ]:
# Neighbors
center = GridPos(ro=1, co=2)
neighbors = grid.neighbors(center)
rprint(f"Neighbors of {center}: {neighbors}")

corner = GridPos(ro=0, co=0)
corner_neighbors = grid.neighbors(corner)
rprint(f"Neighbors of corner {corner}: {corner_neighbors}")

In [ ]:
# Count neighbor pairs
pairs = list(grid.neighbor_pairs())
rprint(f"Total neighbor pairs: {len(pairs)}")
rprint(f"First 5 pairs: {pairs[:5]}")

## 8. PlacementState

Track piece placements on the grid.


In [ ]:
# Create placement state
state = PlacementState(grid)
rprint(f"Initial state: {state}")
rprint(f"Empty positions: {state.empty_count}")

In [ ]:
# Place some pieces
piece_a = PieceId(sheet_id="sheet1", piece_id=0)
piece_b = PieceId(sheet_id="sheet1", piece_id=1)
piece_c = PieceId(sheet_id="sheet2", piece_id=0)

state.place(piece_a, GridPos(ro=0, co=0), Orientation.DEG_0)
state.place(piece_b, GridPos(ro=0, co=1), Orientation.DEG_0)
state.place(piece_c, GridPos(ro=1, co=0), Orientation.DEG_270)

rprint(f"After placing 3 pieces: {state}")

In [ ]:
# Query placements
rprint(f"Piece at (0,0): {state.get_placement(GridPos(ro=0, co=0))}")
rprint(f"Position of piece_b: {state.get_position(piece_b)}")
rprint(f"Placed pieces: {state.placed_pieces()}")

In [ ]:
# Move a piece (placing it elsewhere removes from old position)
state.place(piece_a, GridPos(ro=2, co=2), Orientation.DEG_90)
rprint(f"After moving piece_a to (2,2):")
rprint(f"  Old position (0,0): {state.get_placement(GridPos(ro=0, co=0))}")
rprint(f"  New position (2,2): {state.get_placement(GridPos(ro=2, co=2))}")

In [ ]:
# Clone for branching
clone = state.clone()
clone.place(
    PieceId(sheet_id="sheet3", piece_id=5), GridPos(ro=3, co=4), Orientation.DEG_180
)

rprint(f"Original state: {state.placed_count} pieces")
rprint(f"Cloned state: {clone.placed_count} pieces")

In [ ]:
# Remove a piece
removed = state.remove(GridPos(ro=0, co=1))
rprint(f"Removed: {removed}")
rprint(f"State after removal: {state}")

## Summary

All core grid model components work correctly:

1. **Orientation** - Enum with arithmetic for rotation composition
2. **GridPos** - Hashable position model with `ro`/`co` attributes
3. **PieceType** - Classification based on flat edge count
4. **OrientedPieceType** - Combines type + orientation
5. **Orientation utilities** - Detection, rotation computation, edge mapping
6. **GridModel** - Grid structure with slot types and neighbor iteration
7. **PlacementState** - Bidirectional piece placement tracking

Ready for integration with real pieces and scoring!
